In [1]:
# %pip install transformer_lens
# %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

import torch as t
from torch import Tensor
import circuitsvis as cv
from jaxtyping import Float
from IPython.display import clear_output, display
from transformer_lens import HookedTransformer

t.set_grad_enabled(False)

gpt2 = HookedTransformer.from_pretrained("gpt2")

sentence0 = "When Mary and John went to the shops, John gave a drink to Mary."
sentence1 = "When Mary and John went to the shops, Mary gave a drink to John."
sentence2 = "The cat sat on the mat."
sentences_all = [sentence0, sentence1, sentence2]

names_filter = lambda name: any(name.endswith(f"hook_{s}") for s in ["pattern", "q", "k", "v"])
logits, cache = gpt2.run_with_cache(sentence0, names_filter=names_filter, remove_batch_dim=True)
logits_all, cache_all = gpt2.run_with_cache(sentences_all, names_filter=names_filter)
logits, cache_full = gpt2.run_with_cache(sentence0, remove_batch_dim=True)

tokens = gpt2.to_str_tokens(sentence0)
tokens_all = gpt2.to_str_tokens(sentences_all)


/home/yuj49/anaconda3/envs/llama_factory/lib/python3.8/site-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/yuj49/anaconda3/envs/llama_factory/lib/python3.8/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Loaded pretrained model gpt2 into HookedTransformer


In [7]:
cv.attention.from_cache(
    cache = cache,
    tokens = tokens,
    layers = 0,
)

In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformer_lens import HookedTransformer, HookedTransformerConfig

# 加载经过训练的 GPT-2 模型和配置
model_path = "/data_vault/pittnail/yuj49/DIAYN/generation_student/gpt2_results/checkpoint-960"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 提取模型配置
config = model.config

# 创建 HookedTransformerConfig 对象
hooked_config = HookedTransformerConfig(
    n_layers=config.n_layer,
    n_heads=config.n_head,
    d_model=config.n_embd,
    d_head=config.n_embd // config.n_head,
    d_mlp=config.n_inner,
    d_vocab=config.vocab_size,
    n_ctx=config.n_ctx,
    act_fn=config.activation_function,
    attn_only=False  # 根据需要设置
)

# 创建 HookedTransformer 对象
hooked_model = HookedTransformer(hooked_config)

# 将模型权重转换为字典
state_dict = model.state_dict()

# 加载权重到 HookedTransformer 对象中
hooked_model.load_state_dict(state_dict)

# 现在你可以使用 hooked_model 进行分析了
# 输入文本
text = "This is a sample text to analyze attention patterns."
tokens = tokenizer(text, return_tensors="pt")["input_ids"]

# 获取模型输出并保存激活缓存
with torch.no_grad():
    outputs = hooked_model(tokens, return_type="logits", output_attentions=True)
    cache = hooked_model.cache

# 使用 circuitsvis 可视化注意力模式
import circuitsvis as cv
cv.attention.from_cache(
    cache=cache,
    tokens=tokens,
    layers=0  # 你想要可视化的层
)


RuntimeError: Error(s) in loading state_dict for HookedTransformer:
	Missing key(s) in state_dict: "embed.W_E", "pos_embed.W_pos", "blocks.0.ln1.w", "blocks.0.ln1.b", "blocks.0.ln2.w", "blocks.0.ln2.b", "blocks.0.attn.W_Q", "blocks.0.attn.W_O", "blocks.0.attn.b_Q", "blocks.0.attn.b_O", "blocks.0.attn.W_K", "blocks.0.attn.W_V", "blocks.0.attn.b_K", "blocks.0.attn.b_V", "blocks.0.attn.mask", "blocks.0.attn.IGNORE", "blocks.0.mlp.W_in", "blocks.0.mlp.b_in", "blocks.0.mlp.W_out", "blocks.0.mlp.b_out", "blocks.1.ln1.w", "blocks.1.ln1.b", "blocks.1.ln2.w", "blocks.1.ln2.b", "blocks.1.attn.W_Q", "blocks.1.attn.W_O", "blocks.1.attn.b_Q", "blocks.1.attn.b_O", "blocks.1.attn.W_K", "blocks.1.attn.W_V", "blocks.1.attn.b_K", "blocks.1.attn.b_V", "blocks.1.attn.mask", "blocks.1.attn.IGNORE", "blocks.1.mlp.W_in", "blocks.1.mlp.b_in", "blocks.1.mlp.W_out", "blocks.1.mlp.b_out", "blocks.2.ln1.w", "blocks.2.ln1.b", "blocks.2.ln2.w", "blocks.2.ln2.b", "blocks.2.attn.W_Q", "blocks.2.attn.W_O", "blocks.2.a